In [1]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn

torch.set_printoptions(edgeitems=2)

In [2]:
# 构建数据集
import random

def generateInput(size):
    inputList = []
    
    for i in range(size): 
        inList = []
        for j in range(100):
            item = random.randint(0, 1)
            if item == 0:
                item = -1
            inList.append(item)
        inputList.append(inList)
        
    return inputList

In [3]:
#折射率
def n(omega):
    if(omega>1e14):
        t=100
        a1=4.582
        a2=0.09921
        a3=0.2109
        a4=-0.02194
        b1=0.00000022971
        b2=0.000000052716
        b3=-0.000000049143
        c=88506.25
        t1=t+273.15
        
        w=(2*np.pi*3e8/omega)*1e6
        c1=w*w-(a3+b3*(t1*t1-c))*(a3+b3*(t1*t1-c))
        c2=a1+b1*(t1*t1-c)+(a2+b2*(t1*t1-c))/c1+a4*w*w
        y=np.sqrt(abs(c2))
    else:
        d1=19.9
        d2=44
        d3=4.533e12*2*np.pi
        d4=0.426e12*2*np.pi
        d5=2.9176e13
        
        e=d1+(d2-d1)*d5**2/(d3**2+1.0j*omega*d4-omega**2)
        y=np.real(np.sqrt(abs(e)))

    return y

In [4]:
def kk(f):
    z=f*n(f)/3e8
    return z

In [5]:
def getOutput(inputList):
    outputList = []
    for i in range(len(inputList)):
        G=np.zeros([2,300])
        d=np.zeros([2,100])
        
        d[1,:] = np.array(inputList)[i,:]
        
        for j in range(100):
            d[0,j] = j*3e-6
        
        for k in range(300):
            f_thz=0.1e12+k*0.15e12
            f_nir=3e8/800e-9
            G[0,k]=-kk(2*np.pi*f_nir)+kk(2*np.pi*(f_nir-f_thz))+kk(2*np.pi*f_thz)
        
        for ii in range(300):
            for jj in range(100):
                if jj == 99:
                    break
                G[1,ii]=G[1,ii]+d[1,jj]*np.exp(1.0j*G[0,ii]*d[0,jj+1])-np.exp(1.0j*G[0,ii]*d[0,jj])/(1.0j*G[0,ii])
        
        peak = np.max(np.abs(G[1,:]))
        G_au = np.abs(G[1,:])/peak
        
        max_index = G_au.tolist().index(1)
        
        lo = max_index 
        # 前向搜索
        while G_au[lo] > 0.5 and lo > 0:
            lo -= 1
            if G_au[lo] <= 0.5:
                break 
        
        hi = max_index
        # 后向搜索
        while G_au[hi] > 0.5 and hi < 299:
            hi += 1
            if G_au[hi] <= 0.5:
                break
        
        if(hi==299 and G_au[hi]>0.5)or(lo==0 and G_au[lo]>0.5):
            bw=-1
        else:
            bw = hi - lo
        outputList.append(bw)
    
    return outputList

In [6]:
inputList = generateInput(50)
outputList = getOutput(inputList)

C:\Users\20241\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:21: ComplexWarning: Casting complex values to real discards the imaginary part


In [7]:
inputList = torch.tensor(inputList)
outputList = torch.tensor(outputList).unsqueeze(1)

In [8]:
n_samples = inputList.shape[0]
n_val = int(0.2 * n_samples)
shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

In [9]:
input_train = inputList[train_indices]
output_train = outputList[train_indices]

input_val = inputList[val_indices]
output_val = outputList[val_indices]

In [10]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train) # <1>
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val) # <1>
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 10 == 0:
            print('Epoch {}, Training loss {}, Validation loss {}'.format(
                epoch, float(loss_train), float(loss_val)))

In [11]:
neuron_count = 300

seq_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(100, neuron_count)),
    ('hidden_activation', nn.ReLU()),
    ('output_linear', nn.Linear(neuron_count, 1))
]))

optimizer = optim.SGD(seq_model.parameters(), lr=1e-4)

training_loop(
    n_epochs = 5000, 
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.MSELoss(),
    t_u_train = input_train.float(),
    t_u_val = input_val.float(), 
    t_c_train = output_train.float(),
    t_c_val = output_val.float())

NameError: name 'OrderedDict' is not defined